# 1-click deepfake

accelerated version (in-memory video processing): https://github.com/RichardErkhov/Richard-roop

`main` branch

In [ ]:
#@markdown **GPU check** (free T4, premium V100 or A100)
!nvidia-smi -L

In [ ]:
#@markdown **Setup environment**

%%bash

# pip install -r roop/requirements.txt
# pip uninstall onnxruntime
# pip install onnxruntime-gpu
pip install 'numpy>=1.23.5' tk onnxruntime-gpu opennsfw2 insightface codeformer-pip gfpgan --progress-bar off

curl -sSL -o roop.zip https://github.com/RichardErkhov/Richard-roop/archive/main.zip
unzip -q -o roop.zip
rm roop.zip
mv Richard-roop-main roop

curl -L -o codeformer.zip https://github.com/janvarev/chain-img-plugin-codeformer/archive/main.zip
unzip -q -o codeformer.zip
rm codeformer.zip
mv chain-img-plugin-codeformer-main/plugins/* roop/plugins
rm -rf chain-img-plugin-codeformer-main

mkdir -p roop/options

In [ ]:
#@markdown **download models weights**

%%bash

download_model () {
    wget -P $1 $2 --content-disposition --continue &> /dev/null
}

download_model roop/models https://huggingface.co/deepinsight/inswapper/resolve/main/inswapper_128.onnx
download_model ~/.opennsfw2/weights https://github.com/bhky/opennsfw2/releases/download/v0.1.0/open_nsfw_weights.h5
download_model ~/.insightface/models https://github.com/deepinsight/insightface/releases/download/v0.7/buffalo_l.zip

# not in roop/ because path is incorrectly handled
download_model models https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth
download_model models https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/GFPGANv1.4.pth
download_model gfpgan/weights https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth
download_model gfpgan/weights https://github.com/xinntao/facexlib/releases/download/v0.2.2/parsing_parsenet.pth

codeformer_link='https://github.com/sczhou/CodeFormer/releases/download/v0.1.0'
codeformer_path=$(python -c 'import codeformer as _; print(_.__path__[0])')/weights/facelib
download_model CodeFormer/weights/CodeFormer $codeformer_link/codeformer.pth
download_model CodeFormer/weights/realesrgan $codeformer_link/RealESRGAN_x2plus.pth
download_model CodeFormer/weights/weights/facelib $codeformer_link/detection_Resnet50_Final.pth
download_model CodeFormer/weights/weights/facelib $codeformer_link/parsing_parsenet.pth
download_model $codeformer_path $codeformer_link/detection_Resnet50_Final.pth
download_model $codeformer_path $codeformer_link/parsing_parsenet.pth

cd ~/.insightface/models && unzip -q -o buffalo_l.zip -d buffalo_l

**ATTENTION:** after setup environment, you have to restart colab runtime

without it, the program will likely fail

In [ ]:
#@markdown **Swap face**

#@markdown files name shouldn’t contain non-latin characters

import json

face_image = "photo-1535644258845-185656906071.jpeg"  # @param {type:"string"}
video_file = "yolo.webm"  # @param {type:"string"}
output_vid = "swapped.mp4"  # @param {type:"string"}
keep_fps = True  # @param {type:"boolean"}
swap_all_faces = False  # @param {type:"boolean"}
#@markdown ↳ if disabled, only the leftmost face in each frame is replaced
enhacer_GFPGAN = True  # @param {type:"boolean"}
#@markdown ↳ if enable, enhance swapped faces but processing is slower
enhacer_CodeFormer = False  # @param {type:"boolean"}
#@markdown enhancer with more options (below) but much slower
gpu_threads = 4  #@param {type:"integer"}
#@markdown ↳ to be decreased if crash / out of memory error<br />
#@markdown suggested value for colab free: 8 if disable enhancer, 6 if enable GFPGAN, 4 if enable CodeFormer<br />
#@markdown if crash even with only 1 thread, try restarting runtime

assert not (enhacer_GFPGAN and enhacer_CodeFormer), "choose only 1 enhancer (GFPGAN or CodeFormer) or neither"

keep_fps_opt = "--keep-fps" if keep_fps else ""
all_faces_opt = "--all-faces" if swap_all_faces else ""

default_chain = "facedetect,faceswap"
if enhacer_GFPGAN: default_chain += ",gfpgan"
elif enhacer_CodeFormer: default_chain += ",codeformer"

with open("roop/options/core.json", "w") as write_file:
    json.dump({
        "default_chain": default_chain,
        "init_on_start": "faceswap",
        "is_demo_row_render": False,
        "v": "2.0"
    }, write_file, indent=2)

#@markdown ---
#@markdown *CodeFormer settings (only effective if enable CodeFormer)*
enhance_background = True  # @param {type:"boolean"}
upsample_face = True  # @param {type:"boolean"}
skip_if_no_face = True  # @param {type:"boolean"}
upscale_video = 2  # @param [1, 2, 3, 4]
codeformer_fidelity = 0.8 #@param {type:"slider", min:0, max:1, step:0.05}

if enhacer_CodeFormer:
    with open("roop/options/codeformer.json", "w") as write_file:
        json.dump({
            "background_enhance": enhance_background,
            "codeformer_fidelity": codeformer_fidelity,
            "face_upsample": upsample_face,
            "skip_if_no_face": skip_if_no_face,
            "upscale": int(upscale_video),
            "v": "2.0"
        }, write_file, indent=2)

!OMP_NUM_THREADS=1 TF_CPP_MIN_LOG_LEVEL=2 python roop/run.py -f $face_image -t $video_file -o $output_vid $keep_fps_opt $all_faces_opt --gpu --gpu-threads $gpu_threads